In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn.functional as F
import random

house_path='dataset//selected_feature_num_can_house_distinct__201901_202103.csv'
host_path='dataset//selected_feature_num_can_host_distinct_201901_202103.csv'
host_pic_path='dataset//feature_img_can_host_distinct_201901_202103.csv'
comment_path='new_dataset//selected_feature_can_comment_basic_201901_202103.csv'


house_id_column='HOUSE_ID'
host_id_column='host_id'
host_pic_id_column='host_id'
comment_id_column='comment_id'
neg_comment_id_column='pos_comment_id'
house_continuous_column=['HOUSE_PRICE_DAY','HOUSE_BATHROOM_NUM', 'HOUSE_BEDROOM_NUM', 'HOUSE_BED_NUM', 'HOUSE_DESCRIPTION_LEN', 'HOUSE_ACCOMMODATES',
         'HOUSE_NAME_LEN', 'HOUSE_NEIGHBOUR_OVERVIEW_LEN','HOUSE_REVIEW_SCORE_CLEAN',
         'HOUSE_REVIEW_SCORE_ACCURACY', 'HOUSE_REVIEW_SCORE_CHECKIN', 'HOUSE_REVIEW_SCORE_COMMUNICATION',
         'HOUSE_REVIEW_SCORE_VALUE', 'HOUSE_REVIEW_SCORE_LOCATION','HOUSE_REVIEW_SCORE_RATE']

host_continuous_column=['HOST_ABOUT_LEN', 'HOST_ACCEPTANCE_RATE', 'HOST_HOUSE_NUM', 'HOST_NAME_LEN', 'HOST_RESPONSE_RATE',
         'HOST_VERIFICATION_NUM', 'info_amount', 'readability', 'user_about_t1', 'user_about_t3', 'user_about_t4',
         'user_about_t2','HOST_HOUSE_REVIEW_SCORE_COMMUNICAT_AVG']



house_comment_continuous_column=['house_tran_review_num','house_tran_reviewer_num', 'house_tran_review_1y_num',
                                 'house_tran_review_30d_num',  'house_tran_comment_subjectivity_score_avg',
                                 'house_tran_comment_plarity_score_avg', 'house_tran_review_theme_RoomService_avg',
                                 'house_tran_review_theme_IndoorEnvironment_avg', 'house_tran_review_theme_NeighborFacilities_avg',
                                 'house_tran_review_Transportation_avg', 'house_tran_review_BookingRelus_avg',
                                 'house_tran_review_TouristScenery_avg','house_tran_review_HostServices_avg',
                                 'house_tran_review_theme_HouseFacilities_avg', 'house_tran_review_theme_AccommodExperience_avg',
                                 'house_tran_review_pos_num', 'house_tran_review_neg_num',
                                 'interval_house_first_review', 'interval_house_previous_review']

host_comment_continuous_column=['host_tran_review_num']

house_categorical_column1=['HOUSE_IS_INSTANT_BOOKABLE']
host_categorical_column1=['HOST_HAS_PROFILE_PIC', 'HOST_IS_IDENTITY_VERIFIED', 'HOST_IS_SUPERHOST', 'HOST_NAME_HAS_DIG',
         'HOST_NAME_HAS_ENGLISH']
house_categorical_column2=['HOUSE_PROPERTY_TYPE', 'HOUSE_ROOM_TYPE']
host_categorical_column2=['HOST_RESPONSE_TIME']


npratio=50 #负样本数
train_ratio = 0.6
val_ratio = 0.2
test_ratio = 0.2

num_heads=8

feature_dim=128
max_history_len=15
num_experts=3
num_tasks=2
multi_embedding_dim = 18  # 设置嵌入维度
expert_hidden_units=[[feature_dim,feature_dim*2],[feature_dim*2,feature_dim]]
gate_hidden_units=[[feature_dim,feature_dim//2],[feature_dim//2,num_experts]]
classifier_dnn_hidden_units=[[feature_dim,feature_dim*2],[feature_dim*2,feature_dim]]
weight_dnn_hidden_units=[[feature_dim*3,feature_dim],[feature_dim,2]]
user_dnn_hidden_units=[[feature_dim*2,feature_dim]]
house_dnn_hidden_units=[[feature_dim*2,feature_dim]]
attetion_dnn_hidden_units=[[4*feature_dim,2*feature_dim],[2*feature_dim,feature_dim]]

lr=0.0001
batch_size = 128
t_cont=0.02
num_interests=4
λ=10

In [2]:
train_user_history_path1 = 'new_dataset2//can_history_201801.csv'
val_user_history_path1 = 'new_dataset2//can_history_201805.csv'
test_user_history_path1 = 'new_dataset2//can_history_201806.csv'


train_neg_comment_path1='new_dataset2//new_can_comment_neg_201801_04_50neg.csv'
val_neg_comment_path1='new_dataset2//new_can_comment_neg_201805_50neg.csv'
test_neg_comment_path1='new_dataset2//new_can_comment_neg_201806_50neg.csv'

sclar_path1='new_dataset/all_feature+label_201801_04_50neg.csv'


train_user_history_path2 = 'new_dataset2//can_history_201802_05.csv'
val_user_history_path2 = 'new_dataset2//can_history_201806.csv'
test_user_history_path2 = 'new_dataset2//can_history_201807.csv'


train_neg_comment_path2='new_dataset2//new_can_comment_neg_201802_05_50neg.csv'
val_neg_comment_path2='new_dataset2//new_can_comment_neg_201806_50neg.csv'
test_neg_comment_path2='new_dataset2//new_can_comment_neg_201807_50neg.csv'

sclar_path2='new_dataset2/all_feature+label_201802_05_50neg.csv'

train_user_history_path3 = 'new_dataset2//can_history_201803_06.csv'
val_user_history_path3 = 'new_dataset2//can_history_201807.csv'
test_user_history_path3 = 'new_dataset2//can_history_201808.csv'


train_neg_comment_path3='new_dataset2//new_can_comment_neg_201803_06_50neg.csv'
val_neg_comment_path3='new_dataset2//new_can_comment_neg_201807_50neg.csv'
test_neg_comment_path3='new_dataset2//new_can_comment_neg_201808_50neg.csv'

sclar_path3='new_dataset2/all_feature+label_201803_06_50neg.csv'

train_user_history_path4 = 'new_dataset2//can_history_201804_07.csv'
val_user_history_path4 = 'new_dataset2//can_history_201808.csv'
test_user_history_path4 = 'new_dataset2//can_history_201809.csv'


train_neg_comment_path4='new_dataset2//new_can_comment_neg_201804_07_50neg.csv'
val_neg_comment_path4='new_dataset2//new_can_comment_neg_201808_50neg.csv'
test_neg_comment_path4='new_dataset2//new_can_comment_neg_201809_50neg.csv'

sclar_path4='new_dataset2/all_feature+label_201804_07_50neg.csv'

train_user_history_path5 = 'new_dataset2//can_history_201805_08.csv'
val_user_history_path5 = 'new_dataset2//can_history_201809.csv'
test_user_history_path5 = 'new_dataset2//can_history_201810.csv'


train_neg_comment_path5='new_dataset2//new_can_comment_neg_201805_08_50neg.csv'
val_neg_comment_path5='new_dataset2//new_can_comment_neg_201809_50neg.csv'
test_neg_comment_path5='new_dataset2//new_can_comment_neg_201810_50neg.csv'

sclar_path5='new_dataset2/all_feature+label_201805_08_50neg.csv'

train_user_history_path6 = 'new_dataset2//can_history_201806_09.csv'
val_user_history_path6 = 'new_dataset2//can_history_201810.csv'
test_user_history_path6 = 'new_dataset2//can_history_201811.csv'


train_neg_comment_path6='new_dataset2//new_can_comment_neg_201806_09_50neg.csv'
val_neg_comment_path6='new_dataset2//new_can_comment_neg_201810_50neg.csv'
test_neg_comment_path6='new_dataset2//new_can_comment_neg_201811_50neg.csv'

sclar_path6='new_dataset2/all_feature+label_201806_09_50neg.csv'

train_user_history_path7 = 'new_dataset2//can_history_201807_10.csv'
val_user_history_path7 = 'new_dataset2//can_history_201811.csv'
test_user_history_path7 = 'new_dataset2//can_history_201812.csv'


train_neg_comment_path7='new_dataset2//new_can_comment_neg_201807_10_50neg.csv'
val_neg_comment_path7='new_dataset2//new_can_comment_neg_201811_50neg.csv'
test_neg_comment_path7='new_dataset2//new_can_comment_neg_201812_50neg.csv'

sclar_path7='new_dataset2/all_feature+label_201807_10_50neg.csv'

In [3]:
train_user_history_paths=[train_user_history_path1,train_user_history_path2,train_user_history_path3,train_user_history_path4,train_user_history_path5,train_user_history_path6,train_user_history_path7]
val_user_history_paths=[val_user_history_path1,val_user_history_path2,val_user_history_path3,val_user_history_path4,val_user_history_path5,val_user_history_path6,val_user_history_path7]
test_user_history_paths=[test_user_history_path1,test_user_history_path2,test_user_history_path3,test_user_history_path4,test_user_history_path5,test_user_history_path6,test_user_history_path7]

train_neg_comment_paths=[train_neg_comment_path1,train_neg_comment_path2,train_neg_comment_path3,train_neg_comment_path4,train_neg_comment_path5,train_neg_comment_path6,train_neg_comment_path7]
val_neg_comment_paths=[val_neg_comment_path1,val_neg_comment_path2,val_neg_comment_path3,val_neg_comment_path4,val_neg_comment_path5,val_neg_comment_path6,val_neg_comment_path7]
test_neg_comment_paths=[test_neg_comment_path1,test_neg_comment_path2,test_neg_comment_path3,test_neg_comment_path4,test_neg_comment_path5,test_neg_comment_path6,test_neg_comment_path7]

sclar_paths=[sclar_path1,sclar_path2,sclar_path3,sclar_path4,sclar_path5,sclar_path6,sclar_path7]

In [4]:
#读取house或host的静态特征
def read_house_host(path, id_column, object, continuous_column, categorical_column1, categorical_column2):
    # 从CSV文件中读取房屋信息
    data = pd.read_csv(path)
    # 创建重新编号的字典
    index = {id: new_id for new_id, id in enumerate(data[id_column].unique())}

    # 提取连续特征和类别特征,multi-hot特征
    continuous_features = data[continuous_column]
    categorical_features1 = data[categorical_column1]  # 类别特征列
    categorical_features2 = data[categorical_column2]

    # 使用LabelEncoder将类别特征转换为编号
    label_encoders = {}  # 存储特征列对应的编码器对象
    categorical_encodings = {}  # 存储特征值对应的编号
    for col in categorical_features2.columns:
        le = LabelEncoder()
        categorical_features2.loc[:, col] = le.fit_transform(categorical_features2[col])
        label_encoders[col] = le
        categorical_encodings[col] = dict(zip(le.transform(le.classes_), le.classes_))
    # 得到每个分类特征的类别数
    num_categories_list2 = [len(le.classes_) for le in label_encoders.values()]
    
    
    if object == 'house':  # house处理多热特征，host处理百分比连续特征
        multi_features = data.iloc[:, 10:52]
        #补缺失值，2
        multi_features=multi_features.fillna(2)
        house_multi_matrix = np.array(multi_features)
        # 创建一个与最后一行维度相同的 NaN 数组
        nan_row = np.full((1, house_multi_matrix.shape[1]),2)
        # 将 NaN 数组插入到 house_multi_matrix 的最后一行
        house_multi_matrix = np.concatenate((house_multi_matrix, nan_row), axis=0)
    else:
        # 将连续特征中的百分比特征转换成数值类型
        # 找到包含百分数的列
        percent_columns = ['HOST_ACCEPTANCE_RATE','HOST_RESPONSE_RATE']  
        # 将百分数转换为浮点数
        for column in percent_columns:
            continuous_features[column] = continuous_features[column].str.rstrip('%').astype(float) / 100.0

                        
    #补缺失值，连续值补-2，分类1特征补2，分类2特征不用补
    continuous_features=continuous_features.fillna(-2)
    categorical_features1=categorical_features1.fillna(2)
    
    # 将特征存放到矩阵中，每一列代表一个特征，特征值存放在对应的house编号索引中
    continuous_matrix = np.array(continuous_features)
    categorical_matrix1 = np.array(categorical_features1).astype(int)
    categorical_matrix2 = np.array(categorical_features2)

    # 添加一行-2
    nan_row = np.full((1, continuous_matrix.shape[1]),-2)
    continuous_matrix = np.vstack((continuous_matrix, nan_row))
    # 计算每一列的最大值加1
    new_row1 = np.full(categorical_matrix1.shape[1], 2)
    max_values2 = np.max(categorical_matrix2, axis=0) + 1
    # 将最大值加1的行添加到categorical_matrix1
    categorical_matrix1 = np.vstack((categorical_matrix1,new_row1))
    categorical_matrix2 = np.vstack((categorical_matrix2, max_values2))

    

    if object == 'house':  # house处理多热特征，host处理百分比连续特征
        return index, continuous_matrix, categorical_matrix1, categorical_matrix2, categorical_encodings, house_multi_matrix, num_categories_list2
    else:
        return index, continuous_matrix, categorical_matrix1, categorical_matrix2, categorical_encodings, num_categories_list2

In [5]:
#读取host图片特征
def read_host_pic_features(path):
    data = pd.read_csv(path)
    # 创建重新编号的字典
    index = {id: new_id for new_id, id in enumerate(data['host_id'].unique())}
    # 提取连续特征
    continuous_features =  data.iloc[:, 5:]
    host_pic_continuous_column = data.iloc[:, 5:].columns.tolist()
    
    #补缺失值-2
    continuous_features=continuous_features.fillna(-2)
        
    # 将特征存放到矩阵中，每一列代表一个特征，特征值存放在对应的house编号索引中
    continuous_matrix = np.array(continuous_features)

    #当host没有pic特征时，取最后一行，特征为-2
    # 创建全为nan的一行
    nan_row = np.full((1, continuous_matrix.shape[1]),-2)
    # 在continuous_matrix最后添加zero_row
    continuous_matrix = np.vstack((continuous_matrix,nan_row))

    return index,continuous_matrix,host_pic_continuous_column


In [6]:
#读取house和host以及负样本动态特征
#读取house和host以及负样本动态特征
def read_comment_features(path,id_column,house_continuous_column,host_continuous_column):
    data = pd.read_csv(path)
    # 创建重新编号的字典
    index = {id: new_id for new_id, id in enumerate(data[id_column].unique())}
    # 提取连续特征
    house_continuous_features = data[house_continuous_column]
    host_continuous_features = data[host_continuous_column]
    #补缺失值-2
    house_continuous_features=house_continuous_features.fillna(-2)
    host_continuous_features=host_continuous_features.fillna(-2)
    
    # 将特征存放到矩阵中，每一列代表一个特征，特征值存放在对应的house编号索引中
    house_continuous_matrix = np.array(house_continuous_features)
    host_continuous_matrix = np.array(host_continuous_features)
    
    # 创建一个全为 -2 的行
    nan_row1 = np.full((1, house_continuous_matrix.shape[1]), -2)
    nan_row2 = np.full((1, host_continuous_matrix.shape[1]), -2)
    # 将 nan_row 插入到 host_continuous_matrix 的最后一行
    house_continuous_matrix = np.vstack((house_continuous_matrix, nan_row1))
    host_continuous_matrix = np.vstack((host_continuous_matrix, nan_row2))

    #提取对应的house_id和host_id
    comment_house=np.array(data['listing_id'])
    comment_host = np.array(data['host_id'])
    #最后一行为-1，comment_id不存在时，house_id,host_id为-1
    comment_house = np.append(comment_house, -1)
    comment_host = np.append(comment_host, -1)

    return index, house_continuous_matrix, host_continuous_matrix, comment_house,comment_host

In [7]:
# 转换 history 列为长度为15的数组
def process_history(history,max_history_len):
    if len(history) >= max_history_len:
        processed_history = history[-max_history_len:]
    else:
        processed_history = [-1] * (max_history_len - len(history)) + history
    return processed_history


# 将填充-1的位置标记为True
def create_mask(history):
    mask = [True if item == -1 else False for item in history]
    return mask

#获取所有用户历史记录对应的house、host索引
def user_history(path, comment_house, comment_host, comment_index, house_index, host_index, host_pic_index,max_history_len):
    data = pd.read_csv(path)
    # 将history列的数据转换为列表格式
    data['history'] = data['history'].apply(lambda x: [int(item) for item in x.strip('[]').split()])
    # 获取唯一的 reviewer_id
    unique_reviewer_ids = data['reviewer_id'].unique()

    # 创建 reviewer_id 到编号的映射字典
    user_index = {reviewer_id: idx for idx, reviewer_id in enumerate(unique_reviewer_ids)}
    # 应用处理函数到每个 history
    data['processed_history'] = data['history'].apply(lambda x: process_history(x, max_history_len))  # shape（user_num,history_num）,comment_id,填充为-1
    # 生成整个 history 的矩阵
    history_matrix = np.array(data['processed_history'].tolist())

    # 将 history_matrix 中的 comment_id 转换为 host_id，不存在的设为 -1
    history_matrix_host = []
    history_matrix_house = []
    history_matrix_host_pic = []
    history_matrix_mask = []
    history_matrix_comment = []
    for user_history in history_matrix:
        user_history_host = []
        user_history_host_pic = []
        user_history_house = []
        user_history_mask = []
        user_history_comment = []

        for comment_id in user_history:
            user_history_comment.append(comment_index.get(comment_id, -1))  # comment编号，填充的comment编号为-1
            user_history_host_id = comment_host[comment_index.get(comment_id, -1)]  # host_id
            user_history_house_id = comment_house[comment_index.get(comment_id, -1)]  # house_id
            user_history_host.append(host_index.get(user_history_host_id, -1))  # host表编号
            user_history_host_pic.append(host_pic_index.get(user_history_host_id, -1))  # host_pic表编号
            user_history_house.append(house_index.get(user_history_house_id, -1))  # house表编号

        history_matrix_comment.append(user_history_comment)  # shape（user_num,history_num）,comment编号
        history_matrix_mask.append(create_mask(user_history))# 在填充的位置标记true
        history_matrix_host.append(user_history_host)  # host表编号,shape（user_num,history_num）
        history_matrix_host_pic.append(user_history_host_pic)  # host_pic表编号
        history_matrix_house.append(user_history_house)  # house表编号  shape（user_num,history_num）

    return user_index, history_matrix_comment, history_matrix_host, history_matrix_host_pic, history_matrix_house, history_matrix_mask

In [8]:
#获得测试集正负样本对应的house、host索引
def get_train_input_index(path,comment_path,npratio,house_comment_scaler,host_comment_scaler,comment_index, house_index, host_index, host_pic_index,user_index,comment_house,comment_host,house_comment_tensor,host_comment_tensor):
    neg_data=pd.read_csv(path)   #负样本特征表，train_neg_comment_path
    df_comment=pd.read_csv(comment_path)  #评论表
    #读取负样本动态特征，房子房东id，以及对应的正样本的评论id
    pos_comment_id=[]
    neg_house_comment_features=[]
    neg_host_comment_features=[]
    neg_house_id=[]
    neg_host_id=[]
    grouped_neg_data = neg_data.groupby('pos_comment_id')
    for name, group in grouped_neg_data:
        pos_comment_id.append(name)  #正样本comment_id
        neg_house_comment_matrix=group[house_comment_continuous_column].values[:]
        neg_host_comment_matrix=group[host_comment_continuous_column].values[:]
        
        #先填缺失值，再标准化
        neg_house_comment_matrix = np.nan_to_num(neg_house_comment_matrix, nan=-2)
        neg_host_comment_matrix = np.nan_to_num(neg_host_comment_matrix, nan=-2)
        
        #标准化
        neg_house_comment_matrix = house_comment_scaler.transform(neg_house_comment_matrix)
        neg_host_comment_matrix = host_comment_scaler.transform(neg_host_comment_matrix)
        
        neg_house_comment_features.append(neg_house_comment_matrix)  #负样本房子评论特征(6,特征数)
        neg_host_comment_features.append(neg_host_comment_matrix)#负样本房东评论特征
        neg_house_id.append(group['listing_id'].values[:])  #负样本房子id
        neg_host_id.append(group['host_id'].values[:])
        

    
    #负样本特征转换成tensor
    neg_house_comment_features=torch.tensor(neg_house_comment_features).clone().detach()
    neg_host_comment_features=torch.tensor(neg_host_comment_features).clone().detach()
    
    #根据id，获得负样本房子，房东的index
    neg_house_id=np.array(neg_house_id)
    neg_host_id=np.array(neg_host_id)
    # 使用NumPy的vectorized操作获取所有id对应的索引
    input_neg_house_index= np.vectorize(house_index.get)(neg_house_id)
    input_neg_host_pic_index= np.vectorize(host_pic_index.get,otypes=[int])(neg_host_id,-1)
    input_neg_host_index = np.vectorize(host_index.get, otypes=[int])(neg_host_id, -1)
    
    #根据正样本的评论id，获得房子，房东。comment。reviewer的index
    #获取正样本的评论id对应的index
    input_pos_comment_index=np.vectorize(comment_index.get)(pos_comment_id)
    #获得房子，房东。reviewer的id
    input_pos_house_id=comment_house[input_pos_comment_index]
    input_pos_host_id=comment_host[input_pos_comment_index]
    input_reviewer_id=df_comment['reviewer_id'][input_pos_comment_index].to_numpy()
    #获得房子，房东。reviewer的index
    input_pos_house_index=np.vectorize(house_index.get)(input_pos_house_id)
    input_pos_host_index=np.vectorize(host_index.get)(input_pos_host_id)
    input_pos_host_pic_index=np.vectorize(host_pic_index.get,otypes=[int])(input_pos_host_id,-1)
    input_reviewer_index=np.vectorize(user_index.get)(input_reviewer_id)

    #获得正样本的动态特征
    pos_house_comment_features =house_comment_tensor[input_pos_comment_index]
    pos_host_comment_features =host_comment_tensor[input_pos_comment_index]
    
    #合并正负样本
    #动态特征
    house_comment_features = torch.cat((pos_house_comment_features.unsqueeze(1), neg_house_comment_features), dim=1)
    host_comment_features = torch.cat((pos_host_comment_features.unsqueeze(1), neg_host_comment_features), dim=1)
    #房子，房东的index
    input_house_index=np.concatenate((input_pos_house_index[:, np.newaxis], input_neg_house_index), axis=1)
    input_host_index=np.concatenate((input_pos_host_index[:, np.newaxis], input_neg_host_index), axis=1)
    input_host_pic_index=np.concatenate((input_pos_host_pic_index[:, np.newaxis], input_neg_host_pic_index), axis=1)
    
    label = np.zeros((len(input_house_index), 1 + npratio))
    label[:, 0] = 1
    label= torch.tensor(label)
    
    return pos_comment_id,input_house_index,input_host_index,input_host_pic_index,input_reviewer_index,label,house_comment_features,host_comment_features

In [9]:
#根据house索引，获得house的特征，返回为tensor
def get_house_input(house_continuous_tensor,house_categorical_tensor1,house_categorical_tensor2,house_multi_tensor,input_house_index):
    # 样本编号转换成tensor
    input_house_index = torch.tensor(input_house_index).clone().detach()
    # house
    # continuous_features
    house_continuous_features = house_continuous_tensor[input_house_index]
    # categorical_features1
    house_categorical_features1 = house_categorical_tensor1[input_house_index]
    # categorical_features2
    house_categorical_feature2 = house_categorical_tensor2[input_house_index]
    # multi_features
    house_multi_features = house_multi_tensor[input_house_index]

    return house_continuous_features,house_categorical_features1,house_categorical_feature2,house_multi_features

In [10]:
def get_host_input(host_continuous_tensor,host_categorical_tensor1,host_categorical_tensor2,host_pic_continuous_tensor,input_host_index, input_host_pic_index):
    # host
    # continuous_features
    host_continuous_features = host_continuous_tensor[input_host_index]
    # categorical_features1
    host_categorical_features1 = host_categorical_tensor1[input_host_index]
    # categorical_features2
    host_categorical_features2 = host_categorical_tensor2[input_host_index]
    # host_pic_features
    host_pic_features = host_pic_continuous_tensor[input_host_pic_index]
            
    return host_continuous_features,host_categorical_features1,host_categorical_features2,host_pic_features

In [11]:
#根据样本中的user编号获得对应house、host编号
def get_user_input_index(history_matrix_comment,history_matrix_host,history_matrix_host_pic,history_matrix_house,history_matrix_mask,input_reviewer_index):
    input_history_comment = history_matrix_comment[input_reviewer_index]  # shape(样本数, max_history)
    input_history_host = history_matrix_host[input_reviewer_index]
    input_history_host_pic = history_matrix_host_pic[input_reviewer_index]
    input_history_house = history_matrix_house[input_reviewer_index]
    input_history_mask = history_matrix_mask[input_reviewer_index]

    return input_history_comment,input_history_host,input_history_host_pic,input_history_house,input_history_mask

In [12]:
#获取user历史house、host的动态特征（只有正样本）
def get_history_comment_input(history_comment_index,house_comment_tensor,host_comment_tensor):
    # 样本编号转换成tensor
    history_comment_index = torch.tensor(history_comment_index).clone().detach()
    house_comment_features=house_comment_tensor[history_comment_index]
    host_comment_features=host_comment_tensor[history_comment_index]
    return house_comment_features,host_comment_features

In [13]:
def creat_scaler(path,column):
    data=pd.read_csv(path)
    continuous_features=data[column]
    # 补缺失值-2
    continuous_features= continuous_features.fillna(-2)
    continuous_matrix=np.array(continuous_features)
    # 创建一个StandardScaler对象
    scaler = StandardScaler()
    scaler.fit(continuous_matrix)
    return scaler

In [14]:
import math

import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import functional as F

#dnn层,hidden_units=[[输入1，输出1],[输入2，输出2]]
class DNN(nn.Module):
    def __init__(self, hidden_units, activation, l2_reg, dropout_rate, use_bn):
        super(DNN, self).__init__()

        # 创建一个空的列表用于存储神经网络的层
        layers = []

        # 遍历每个隐藏层的设置
        for hidden_size in hidden_units:
            # 添加一个线性层（全连接层），输入维度为 hidden_size[0]，输出维度为 hidden_size[1]
            layers.append(nn.Linear(hidden_size[0], hidden_size[1]))

            # 如果 use_bn 为 True，则添加批归一化层
            if use_bn:
                layers.append(nn.BatchNorm1d(hidden_size[1]))

            # 添加激活函数层，激活函数为传入的 activation 函数
            layers.append(activation)

            # 添加 dropout 层，dropout_rate 为 dropout 的比例
            layers.append(nn.Dropout(dropout_rate))

        # 将所有层组合成一个顺序的神经网络
        self.dnn_network = nn.Sequential(*layers)

    def forward(self, x):
        # 在前向传播过程中，输入数据 x 经过 dnn_network 这个神经网络模型
        return self.dnn_network(x)

#用户建模
#用户建模
class DinAttention(nn.Module):
    def __init__(self,attetion_dnn_hidden_units):
        super(DinAttention, self).__init__()
        self.dnn1= DNN(attetion_dnn_hidden_units, nn.Sigmoid(), 0.0, 0.0, False)
        self.dnn2= nn.Linear(feature_dim, 1)


    def forward(self, history_matrix, house,mask=None):
        query= torch.unsqueeze(house, dim=-2)#(batch,npratio+1,1,400)
        keys=torch.unsqueeze(history_matrix, dim=1)  #(batch,1,15,400)
        query_len=query.size()[1]  #npratio+1
#         keys = keys.repeat(1,query_len,1,1)#(batch,npratio+1,15,400)
        keys = keys.expand(-1,query_len ,-1, -1)
        keys_len = keys.size()[2]   #15
#         querys = query.repeat(1,1, keys_len, 1)#(batch,npratio+1,15,400)
        querys = query.expand(-1,-1, keys_len, -1)
        atten_input = torch.cat([querys, keys, querys - keys, querys * keys], dim=-1)  # (batch,npratio+1,T, 4 * embed_size)
        
        # 经过三层全连接层
        output1 = self.dnn1(atten_input)
        output2 = self.dnn2(output1)  # (batch,npratio+1,T, 1)
#         outputs = output2.transpose(2, 3)  # (batch,npratio+1, 1,T)
#         user_att = outputs.view(-1, keys_len)  # 展平为一维向量（batch，15）
        user_att = output2.squeeze(-1) #每个历史记录的权重(batch,npratio+1,T)
        
        if mask is not None:
            #mask(batch,T),扩展成(batch,npratio+1,T)
            mask = mask.unsqueeze(1)
            mask = mask.repeat(1,query_len,1)
            user_att = user_att.masked_fill(mask, float('-inf'))  # 对填充向量进行mask操作(batch,npratio+1,T)
        user_att = user_att / (keys.size()[-1] ** 0.5)
        user_att = torch.softmax(user_att, dim=-1)  # (batch,npratio+1,T)
        weighted_sum = torch.matmul(user_att.unsqueeze(-2), keys).squeeze(-2)  # (batch,npratio+1,400)
        return weighted_sum

 # 全连接层
def dense_layer(in_features, out_features):
    # in_features=hidden_size,out_features=1
    return nn.Sequential(
        nn.Linear(in_features, out_features,bias=True),
        nn.ReLU())


def categorical1_embedding(num_categorical1_features): #输入categorical1的特征数
    # 创建一个列表来存储每个嵌入层
    categorical1_embeddings = []
    for i in range(num_categorical1_features):
        embedding_layer1 = nn.Embedding(3, feature_dim)
        categorical1_embeddings.append(embedding_layer1)
    return categorical1_embeddings

def categorical2_embedding(num_categorical2_features,num_categories_list2): #输入categorical2的特征数，各个特征的特征值数量的列表
    # 创建一个列表来存储每个嵌入层
    categorical2_embeddings = []
    # 循环遍历每个特征，为每个特征创建嵌入层并添加到列表中
    for i in range(num_categorical2_features):
        categorie2_embedding_dim = int(math.log2(num_categories_list2[i]+1))  # 嵌入维度为类别数的底数为2的对数
        embedding_layer2 = nn.Embedding(num_categories_list2[i]+1, feature_dim)
        categorical2_embeddings.append(embedding_layer2)
    return categorical2_embeddings

def continuous_embedding(num_continuous_features):
    continuous_embedding_layers = []
    for i in range(num_continuous_features):
        embedding_layer = dense_layer(1, feature_dim)
        continuous_embedding_layers.append(embedding_layer)
    return continuous_embedding_layers


    




class HouseEmbedding(nn.Module):
    def __init__(self,num_house_categorical1_features,num_house_categorical2_features,num_house_categories_list2,
                 num_house_multi_features,multi_embedding_dim,num_host_categorical1_features,num_host_categorical2_features,
                 num_host_categories_list2,num_house_continuous_features,num_host_continuous_features,
                 num_host_pic_continuous_features,num_house_comment_features,feature_dim):
        super(HouseEmbedding, self).__init__()
        self.multi_embedding_dim=multi_embedding_dim
        self.num_house_multi_features=num_house_multi_features
        self.sum_num_features=num_house_categorical1_features+num_house_categorical2_features+num_house_continuous_features+num_house_multi_features+num_house_comment_features+num_host_categorical1_features+num_host_categorical2_features+num_host_continuous_features+num_host_pic_continuous_features
        # house_categorical1
        self.house_categorical1_embeddings = categorical1_embedding(num_house_categorical1_features)
        #house_categorical2
        self.house_categorical2_embeddings = categorical2_embedding(num_house_categorical2_features,num_house_categories_list2)
        #house_continuous
        self.house_continuous_embedding_layer = continuous_embedding(num_house_continuous_features)
        #house_multi
        self.multi_embedding_layer = categorical1_embedding(num_house_multi_features)
        #house_comment
        self.house_comment_embedding_layer = continuous_embedding(num_house_comment_features)

        # host_categorical1
        self.host_categorical1_embeddings = categorical1_embedding(num_host_categorical1_features)
        # host_categorical2
        self.host_categorical2_embeddings = categorical2_embedding(num_host_categorical2_features,num_host_categories_list2)
        #host_continuous_features
        self.host_continuous_embedding_layer = continuous_embedding(num_host_continuous_features)
        #host_pic_features
        self.host_pic_embedding_layer = continuous_embedding(num_host_pic_continuous_features)
        
        # MLP256-11
        self.house_dense_layer = dense_layer(self.sum_num_features*feature_dim,feature_dim)

    def forward(self, house_continuous_features,house_categorical_features1,house_categorical_feature2,house_multi_features,house_comment_features,host_continuous_features, host_categorical_features1, host_categorical_features2,host_comment_features, host_pic_features):
        # house
        # house_continuous_features(样本数, 2, 14)
        # house_multi_features(样本数, 2, 42->18),(样本数, 15, 42->18)
        # 对多值特征进行嵌入
        house_multi_embeddings = torch.cat(
            [embedding_layer(house_multi_features[:,:, i]) for i, embedding_layer in
             enumerate(self.multi_embedding_layer)], dim=-1)  
        # house_comment_features（样本数, 2, 18）
        # house_categories_embeddings1,shape(样本数，2,2(2*1))
        house_categories_embeddings1 = torch.cat(
            [embedding_layer(house_categorical_features1[:,:, i]) for i, embedding_layer in
             enumerate(self.house_categorical1_embeddings)], dim=-1)
        # house_categories_embeddings2，shape(样本数，2,(52+4)->embedding维度之和=7(5+2))
        house_categories_embeddings2 = torch.cat(
            [embedding_layer(house_categorical_feature2[:, :, i]) for i, embedding_layer in
             enumerate(self.house_categorical2_embeddings)], dim=-1)
        #连续值embedding(样本数，2,特征数,200)
        house_continuous_embeddings = torch.cat(
            [embedding_layer(house_continuous_features[:,:, i].unsqueeze(2).float()) for i, embedding_layer in
             enumerate(self.house_continuous_embedding_layer)], dim=-1)
        # house_comment_features（样本数, 2, 19,200）
        house_comment_embeddings = torch.cat(
            [embedding_layer(house_comment_features[:,:, i].unsqueeze(2).float()) for i, embedding_layer in
             enumerate(self.house_comment_embedding_layer)], dim=-1)
        
        # concate（样本数, 2,61）
        house_con = torch.cat(
            [house_continuous_embeddings, house_multi_embeddings, house_comment_embeddings, house_categories_embeddings1,
             house_categories_embeddings2], dim=-1)
        
        # host_categories_embeddings1，shape(样本数，2,10(2*5))
        host_categories_embeddings1 = torch.cat(
            [embedding_layer(host_categorical_features1[:, :, i]) for i, embedding_layer in
             enumerate(self.host_categorical1_embeddings)], dim=-1)
        #host_categories_embeddings2，shape(样本数，2,5->2(2*1))
        host_categories_embeddings2 = torch.cat(
            [embedding_layer(host_categorical_features2[:, :, i]) for i, embedding_layer in
             enumerate(self.host_categorical2_embeddings)], dim=-1)
        #host_continuous_features
        host_continuous_embeddings = torch.cat(
            [embedding_layer(host_continuous_features[:,:, i].unsqueeze(2).float()) for i, embedding_layer in
             enumerate(self.host_continuous_embedding_layer)], dim=-1)
        #host_pic_features
        host_pic_embeddings = torch.cat(
            [embedding_layer(host_pic_features[:,:, i].unsqueeze(2).float()) for i, embedding_layer in
             enumerate(self.host_pic_embedding_layer)], dim=-1)        
    
        # concate(样本数，2,195)-host_comment_features(11)=184
        host_con = torch.cat(
            [host_continuous_embeddings,host_pic_embeddings,host_categories_embeddings1,host_categories_embeddings2], dim=-1)
        
        all_con=torch.cat(
            [house_con,host_con], dim=-1)
        
        house_vec = self.house_dense_layer(all_con.float())  # shape(样本数, 2,200)
        return house_vec


In [15]:
class MatchingModel(nn.Module):
    def __init__(self, num_house_categorical1_features,num_host_categorical1_features,num_house_categorical2_features,num_host_categorical2_features,num_house_multi_features,multi_embedding_dim,
                 num_house_categories_list2,num_host_categories_list2,num_house_continuous_features,num_host_continuous_features,
                 num_host_pic_continuous_features,num_house_comment_features,feature_dim,num_heads,max_history_len,
                 num_experts, num_tasks, expert_hidden_units, gate_hidden_units,user_dnn_hidden_units,house_dnn_hidden_units,attetion_dnn_hidden_units,device):
        super(MatchingModel, self).__init__()
        #embedding
        self.house_embedding=HouseEmbedding(num_house_categorical1_features,num_house_categorical2_features,num_house_categories_list2,
                                            num_house_multi_features,multi_embedding_dim,num_host_categorical1_features,
                                            num_host_categorical2_features,num_host_categories_list2,num_house_continuous_features,
                                            num_host_continuous_features,num_host_pic_continuous_features,num_house_comment_features,
                                            feature_dim)

        self.interest_num = num_interests
        self.num_heads =num_interests
        self.device=device
        self.seq_len=max_history_len
        self.hidden_size=feature_dim
        self.position_embedding = nn.Parameter(torch.randn(1, self.seq_len, self.hidden_size))  #（1，T,dim)
        self.linear1 = nn.Sequential(
                        nn.Linear(self.hidden_size, self.hidden_size * 4, bias=False),    #tanh(linear(dim,dim*4))  
                        nn.Tanh()
                    )
        self.linear2 = nn.Linear(self.hidden_size * 4, self.num_heads, bias=False)  #（dim*4,兴趣数）
        

    def forward(self,house_continuous_features,house_categorical_features1,house_categorical_feature2,house_multi_features,house_comment_features,
                 host_continuous_features, host_categorical_features1, host_categorical_features2,host_comment_features, host_pic_features,
                 user_house_continuous_features,user_house_categorical_features1,user_house_categorical_feature2,user_house_multi_features,user_house_comment_features,
                 user_host_continuous_features,user_host_categorical_features1,user_host_categorical_features2,user_host_comment_features,user_host_pic_features
                ,mask):
        #house
        # shape(样本数,npratio+1,200)
        house_vec=self.house_embedding(house_continuous_features,house_categorical_features1,house_categorical_feature2,house_multi_features,house_comment_features,host_continuous_features, host_categorical_features1, host_categorical_features2,host_comment_features, host_pic_features)
        # shape(样本数, 15,200)
        user_house_emb=self.house_embedding(user_house_continuous_features,user_house_categorical_features1,user_house_categorical_feature2,user_house_multi_features,user_house_comment_features,user_host_continuous_features,user_host_categorical_features1,user_host_categorical_features2,user_host_comment_features,user_host_pic_features)

        # 位置嵌入堆叠一个batch，然后与历史物品嵌入相加
        user_house_emb_add_pos = user_house_emb + self.position_embedding.repeat(user_house_emb.shape[0], 1, 1)
        # shape=(batch_size, maxlen, hidden_size*4)
        user_house_hidden = self.linear1(user_house_emb_add_pos)
        # shape=(batch_size, maxlen, num_heads)
        att_w  = self.linear2(user_house_hidden)
        # shape=(batch_size, num_heads, maxlen)
        att_w  = torch.transpose(att_w, 2, 1).contiguous()
        
        #mask后再做softmax
        atten_mask = torch.unsqueeze(mask, dim=1).repeat(1, self.num_heads, 1)  # shape=(batch_size, num_heads, maxlen)
        paddings = torch.ones_like(atten_mask, dtype=torch.float) * (-2 ** 32 + 1) # softmax之后无限接近于0
        
        item_att_w = torch.where(atten_mask.bool(), paddings, att_w)  #torch.eq(atten_mask, 0)当mak中为0时，返回true，即mask为0，填充负无穷
        item_att_w = F.softmax(item_att_w, dim=-1) # 矩阵A，shape=(batch_size, num_heads, maxlen)
        
        # interest_emb即论文中的Vu
        interest_emb = torch.matmul(item_att_w, # shape=(batch_size, num_heads, maxlen)
                                user_house_emb_add_pos # shape=(batch_size, maxlen, embedding_dim)
                                ) # shape=(batch_size, num_heads, embedding_dim)
        #兴趣矩阵*item=（batch，1，兴趣数，dim）*（batch，正负样本，dim，1）
        multi_score = torch.matmul(interest_emb.unsqueeze(1),house_vec.unsqueeze(-1))  #（batch,正负样本,proposals_num.1)
            
        #分数最大的兴趣索引
        user_embedding_idx = torch.argmax(multi_score, dim=2) #（batch,正负样本，1）
        #与正样本点积最大的兴趣向量索引
        user_embedding_idx=user_embedding_idx[:,0,0] #（batch）
        user_embedding_idx = user_embedding_idx.unsqueeze(1)  # 将维度扩展为（batch，1）
        user_embedding_idx = user_embedding_idx.unsqueeze(2)  # 将维度扩展为（batch，1，1）
        #在兴趣向量矩阵中找到索引对应的兴趣向量作为用户向量（batch，1，dim）
        user_embedding= torch.gather(interest_emb, 1, user_embedding_idx.expand(len(user_embedding_idx), 1, feature_dim))
        pos_house_vec=house_vec[:,0,:] .unsqueeze(1) #(batch,1,dim)
        neg_house_vec=house_vec[:,1:,:]
        #用于loss的分数(batch,1,1,dim)*#(batch,1,dim,1)
        pos_score= torch.matmul(user_embedding.unsqueeze(1),pos_house_vec.unsqueeze(-1)).squeeze(-2).squeeze(-1) #(batch,1)
        neg_score= torch.matmul(user_embedding.unsqueeze(1),neg_house_vec.unsqueeze(-1)).squeeze(-2).squeeze(-1) #(batch,负样本)

        #用于排序的分数 (batch,正负样本)
        score, _=torch.max(multi_score,dim=2)
        score=score.squeeze(-1)  #(batch,正负样本)
        
        
            
        return score,pos_score,neg_score,item_att_w
            


In [16]:
def calculate_atten_loss(attention):
    #attention=item_att_w(batch_size, num_heads, maxlen)
    C_mean = torch.mean(attention, dim=2, keepdim=True) #(batch_size, num_heads)
    C_reg = (attention - C_mean)  #(batch_size, num_heads, maxlen)
    # C_reg = C_reg.matmul(C_reg.transpose(1,2)) / self.hidden_size
    C_reg = torch.bmm(C_reg, C_reg.transpose(1, 2))/feature_dim  #(batch_size, num_heads, num_heads)
    dr = torch.diagonal(C_reg, dim1=-2, dim2=-1) #(batch_size, num_heads)
    n2 = torch.norm(dr, dim=(1)) ** 2
    return n2.sum()

In [17]:
from torch.utils.data import Dataset, DataLoader, TensorDataset



class BPRLoss(nn.Module):
    def __init__(self):
        super(BPRLoss, self).__init__()

    def forward(self, positive_scores, negative_scores):
        # 计算正样本和负样本之间的分数差异
        loss = -torch.log(torch.sigmoid(positive_scores - negative_scores))
        return torch.mean(loss)




In [18]:
def model_training(model,train_loader,val_loader,
                   rec_criterion,optimizer,EPOCH,device):
    # 定义早停策略的参数
    best_val_loss = float('inf')  # 初始化最佳验证损失为正无穷
    patience = 3  # 容忍多少个epoch没有验证性能提升
    early_stopping_counter = 0  # 初始化计数器
    for epoch in range(EPOCH):
        model.train()  # 设置模型为训练模式
        total_classfier_loss = 0.0
        total_rec_loss = 0.0
        total_loss = 0.0
        total_loss_attend = 0.0
        total_loss_contrastive = 0.0
        total_loss_construct = 0.0
        for batch in train_loader:
            batch = [data.to(device) for data in batch]
            house_continuous_features, house_categorical_features1, house_categorical_feature2, house_multi_features, house_comment_features, \
            host_continuous_features, host_categorical_features1, host_categorical_features2, host_comment_features, host_pic_features, \
            user_house_continuous_features, user_house_categorical_features1, user_house_categorical_feature2, user_house_multi_features, user_house_comment_features, \
            user_host_continuous_features, user_host_categorical_features1, user_host_categorical_features2, user_host_comment_features, user_host_pic_features, \
            label, mask = batch

            for param in model.parameters():
                param.requires_grad = True
            optimizer.zero_grad()
            score,pos_score,neg_score,item_att_w= model(house_continuous_features,
                                                                   house_categorical_features1,
                                                                   house_categorical_feature2, house_multi_features,
                                                                   house_comment_features,
                                                                   host_continuous_features, host_categorical_features1,
                                                                   host_categorical_features2, host_comment_features,
                                                                   host_pic_features,
                                                                   user_house_continuous_features,
                                                                   user_house_categorical_features1,
                                                                   user_house_categorical_feature2,
                                                                   user_house_multi_features,
                                                                   user_house_comment_features,
                                                                   user_host_continuous_features,
                                                                   user_host_categorical_features1,
                                                                   user_host_categorical_features2,
                                                                   user_host_comment_features, user_host_pic_features
                                                                   , mask)
            

            
            #推荐损失
            softmax_score=torch.softmax(score,dim=1)  #(batch,正负样本)
            rec_loss=softmax_score[:,0]  #正样本分数
            rec_loss = torch.mean(-torch.log(rec_loss))
            reg_loss=calculate_atten_loss(item_att_w)
            loss = rec_loss+λ*reg_loss
            loss.backward()
            optimizer.step()
            
            #计算平均loss

            total_loss+=loss.item()
        

        average_loss=total_loss/len(train_loader)
        
        if (epoch+1)%5==0:
            print(f"Epoch {epoch + 1},loss:{average_loss}")
            
            # 验证集评估
            model.eval()  # 将模型切换为评估模式
            with torch.no_grad():  # 在评估模式下不计算梯度
                total_loss_val = 0.0
                total_val_auc = 0.0
                total_regu_loss_val=0.0
                total_rec_loss_val = 0.0
                total_loss_attend_val = 0.0
                total_loss_contrastive_val = 0.0
                total_loss_construct_val = 0.0
                for batch_val in val_loader:  # 假设你有一个名为 val_loader 的验证集数据加载器
                    batch_val = [data.to(device) for data in batch_val]
                    house_continuous_features_val, house_categorical_features1_val, house_categorical_feature2_val, house_multi_features_val, house_comment_features_val, \
                    host_continuous_features_val, host_categorical_features1_val, host_categorical_features2_val, host_comment_features_val, host_pic_features_val, \
                    user_house_continuous_features_val, user_house_categorical_features1_val, user_house_categorical_feature2_val, user_house_multi_features_val, user_house_comment_features_val, \
                    user_host_continuous_features_val, user_host_categorical_features1_val, user_host_categorical_features2_val, user_host_comment_features_val, user_host_pic_features_val, \
                    label_val, mask_val = batch_val
                    #在验证集上进行前向传播
                    score_val,pos_score_val,neg_score_val,item_att_w_val= model(house_continuous_features_val, house_categorical_features1_val,
                                                    house_categorical_feature2_val, house_multi_features_val,
                                                    house_comment_features_val, host_continuous_features_val,
                                                    host_categorical_features1_val, host_categorical_features2_val,
                                                    host_comment_features_val, host_pic_features_val,
                                                    user_house_continuous_features_val,
                                                    user_house_categorical_features1_val,
                                                    user_house_categorical_feature2_val, user_house_multi_features_val,
                                                    user_house_comment_features_val, user_host_continuous_features_val,
                                                    user_host_categorical_features1_val, user_host_categorical_features2_val,
                                                    user_host_comment_features_val, user_host_pic_features_val,
                                                    mask_val)

                    
                    #推荐损失
                    softmax_score_val=torch.softmax(score_val,dim=1)  #(batch,正负样本)
                    rec_loss_val=softmax_score_val[:,0]
                    rec_loss_val = torch.mean(-torch.log(rec_loss_val))
                    reg_loss_val=calculate_atten_loss(item_att_w_val)
                    loss_val = rec_loss_val+λ*reg_loss_val
                    #总损失

                    total_loss_val+=loss_val.item()

                    # 计算验证集上的AUC
                    positive_scores_val = score_val[:, 0]
                    positive_scores_val = positive_scores_val.unsqueeze(-1)
                    negative_scores_val = score_val[:, 1:]
                    predictions_val = (positive_scores_val > negative_scores_val).float()
                    equal_indices_val = positive_scores_val == negative_scores_val
                    predictions_val[equal_indices_val] = 0.5
                    correct_val = predictions_val.sum().item()
                    total_val_auc += correct_val / (len(predictions_val)*npratio)
                    
                average_val_loss = total_loss_val / len(val_loader)
                average_auc_val=total_val_auc/ len(val_loader)

                print(f"Validation Loss: {average_val_loss},AUC: {average_auc_val}")
                if average_val_loss<best_val_loss:
                    best_val_loss = average_val_loss
                    early_stopping_counter = 0
                else:
                    early_stopping_counter += 1
                if early_stopping_counter >= patience:
                    print(f"早停策略触发，停止训练在第 {epoch} 个epoch.")
                    break






In [19]:
# model_training(model,train_loader,val_loader,classfier,house_embedding,host_embedding,classfier_optimizer,classfier_criterion,att_criterion,rec_criterion,optimizer,500,device)

In [20]:
def create_centered_distance(X):
    # 计算每个样本的平方模长
    r = torch.sum(X ** 2, dim=1, keepdim=True)

    # 计算欧氏距离矩阵
    D = torch.sqrt(torch.clamp(r - 2 * torch.mm(X, X.t()) + r.t(), min=0.0) + 1e-8)

    # 计算中心化距离矩阵
    D -= torch.mean(D, dim=0, keepdim=True)
    D -= torch.mean(D, dim=1, keepdim=True)
    D += torch.mean(D)
    return D
def create_distance_covariance(D1, D2):
    # 计算 D1 和 D2 之间的距离协方差
    n_samples = float(D1.size(0))
    dcov = torch.sqrt(torch.clamp(torch.sum(D1 * D2) / (n_samples * n_samples), min=0.0) + 1e-8)
    return dcov
def create_distance_correlation( X1, X2):
    X1=X1.reshape(-1,feature_dim)
    X2=X2.reshape(-1,feature_dim)
    D1 = create_centered_distance(X1)
    D2 = create_centered_distance(X2)

    dcov_12 = create_distance_covariance(D1, D2)
    dcov_11 = create_distance_covariance(D1, D1)
    dcov_22 = create_distance_covariance(D2, D2)

    dcor = dcov_12 / (torch.sqrt(torch.clamp(dcov_11 * dcov_22, min=0.0)) + 1e-10)
    return dcor

In [21]:
def test_model(model, test_loader):
    model.eval()  # 设置模型为评估模式
    with torch.no_grad():  # 在评估模式下不计算梯度
        total_loss_test = 0.0
        total_rec_loss_test = 0.0
        total_test_auc=0.0
        total_loss_attend_test = 0.0
        total_loss_contrastive_test = 0.0
        total_loss_construct_test = 0.0
        total_distance_correlation=0.0
        results = []  # 用于保存结果的列表
        for batch_test in test_loader:  # 假设你有一个名为 val_loader 的验证集数据加载器
            batch_test = [data.to(device) for data in batch_test]
            pos_comment_id,house,host,reviewer,house_continuous_features_test, house_categorical_features1_test, house_categorical_feature2_test, house_multi_features_test, house_comment_features_test, \
            host_continuous_features_test, host_categorical_features1_test, host_categorical_features2_test, host_comment_features_test, host_pic_features_test, \
            user_house_continuous_features_test, user_house_categorical_features1_test, user_house_categorical_feature2_test, user_house_multi_features_test, user_house_comment_features_test, \
            user_host_continuous_features_test, user_host_categorical_features1_test, user_host_categorical_features2_test, user_host_comment_features_test, user_host_pic_features_test, \
            label_test, mask_test = batch_test
                    
            score_test,pos_score_test,neg_score_test,item_att_w_test= model(house_continuous_features_test, house_categorical_features1_test,
                                                                                                                            house_categorical_feature2_test, house_multi_features_test, 
                                                                                                                            house_comment_features_test, host_continuous_features_test, 
                                                                                                                            host_categorical_features1_test, host_categorical_features2_test, 
                                                                                                                            host_comment_features_test, host_pic_features_test,                                                                                           
                                                                                                                            user_house_continuous_features_test, user_house_categorical_features1_test, 
                                                                                                                            user_house_categorical_feature2_test, user_house_multi_features_test,
                                                                                                                            user_house_comment_features_test, user_host_continuous_features_test, 
                                                                                                                            user_host_categorical_features1_test, user_host_categorical_features2_test, 
                                                                                                                            user_host_comment_features_test, user_host_pic_features_test,mask_test)

            
            #推荐损失
            softmax_score_test=torch.softmax(score_test,dim=1)  #(batch,正负样本)
            rec_loss_test=softmax_score_test[:,0]
            rec_loss_test = torch.mean(-torch.log(rec_loss_test))
            reg_loss_test=calculate_atten_loss(item_att_w_test)
            loss_test = rec_loss_test+λ*reg_loss_test
            #总损失

            total_loss_test+=loss_test.item()
            
            
            # 计算验证集上的AUC
            positive_scores_test = score_test[:, 0]
            positive_scores_test = positive_scores_test.unsqueeze(-1)
            negative_scores_test = score_test[:, 1:]
            predictions_test = (positive_scores_test > negative_scores_test).float()
            equal_indices = positive_scores_test == negative_scores_test
            predictions_test[equal_indices] = 0.5
            correct_test = predictions_test.sum().item()
            total_test_auc += correct_test /  (len(predictions_test)*npratio)

        
            for i in range(house.shape[0]):
                #正样本
                results.append({
                    'pos_comment_id':pos_comment_id[i],
                    'house': house[i][0],
                    'host': host[i][0],
                    'reviewer': reviewer[i],
                    'score': score_test[i][0],  # 正样本分数

                    'label': label_test[i][0]
                    
                })
                for j in range(npratio):
                    #负样本
                    results.append({
                        'pos_comment_id':pos_comment_id[i],
                        'house': house[i][1+j],
                        'host': host[i][1+j],
                        'reviewer': reviewer[i],             
                        'score': score_test[i][1+j],  # 负样本分数

                        'label': label_test[i][1+j]
                    })
                
        average_test_loss = total_loss_test / len(test_loader)
        average_auc_test=total_test_auc/ len(test_loader)
        average_rec_loss_test=total_rec_loss_test/ len(test_loader)

        print(f"Test Loss: {average_test_loss},AUC: {average_auc_test},rec_loss: {average_rec_loss_test}")
        
        # 将结果列表转换为DataFrame
        df_results = pd.DataFrame(results)
        df_results['pos_comment_id'] = df_results['pos_comment_id'].apply(lambda x: x.item())
        df_results['house'] = df_results['house'].apply(lambda x: x.item())
        df_results['host'] = df_results['host'].apply(lambda x: x.item())
        df_results['reviewer'] = df_results['reviewer'].apply(lambda x: x.item())
        df_results['score'] = df_results['score'].apply(lambda x: x.item())

        df_results['label'] = df_results['label'].apply(lambda x: x.item())
        df_results['item_count'] = df_results.groupby('reviewer')['reviewer'].transform('count')
        
  
        
        
        return df_results,average_test_loss,average_auc_test   

In [22]:
# df_results,average_test_loss,average_auc_test,average_att_loss_test=test_model(model, test_loader)
# print(df_results)

In [23]:
#top-n
def top_evaluation(df_results,n):
    df_results_shuffled = df_results.sample(frac=1).reset_index(drop=True)
    # 初始化结果列表
    ndcg_list = []
    hr_list=[]
    
    for user_id, user_group in df_results_shuffled.groupby('pos_comment_id'):
        user_group = user_group.sort_values(by='score', ascending=False)  # 按分数降序排序
        top_items = user_group.head(n)['label'].tolist()  # 选取前n个的标签
        true_labels = user_group['label'].tolist()  # 用户所有候选物品的真实标签
                
        # 计算NDCG
        idcg = sorted(true_labels, reverse=True)[:n]  # 理想情况下的DCG
        dcg = sum(rel / np.log(i + 2) for i, rel in enumerate(top_items))  # 计算DCG
        ndcg = dcg / sum(rel / np.log(i + 2) for i, rel in enumerate(idcg))  # 计算NDCG
        ndcg_list.append(ndcg)
        # 计算HR@N
        hit_rate = 1 if any(label == 1 for label in top_items) else 0  # 若Top N中有真实正样本，HR@N为1，否则为0
        hr_list.append(hit_rate)

        
    # 计算平均值
    average_ndcg = np.mean(ndcg_list)
    average_hr = np.mean(hr_list)
    
    return average_ndcg, average_hr

In [ ]:
import cloudpickle
# 创建一个空的DataFrame来存储结果
test_auc_df = pd.DataFrame(columns=['时间窗','实验数', '测试集总损失', 'AUC','NDCG@5','HR@5','NDCG@10','HR@10'])
for train_user_history_path,val_user_history_path,test_user_history_path,train_neg_comment_path,val_neg_comment_path,test_neg_comment_path,sclar_path in zip(train_user_history_paths,val_user_history_paths,test_user_history_paths,train_neg_comment_paths,val_neg_comment_paths,test_neg_comment_paths,sclar_paths):
    
    for n in range(5):
        print(f'i=+{n+1}')

        # 假设文件名总是有相同的结构，可以根据固定的位置进行切片
        start_index = train_user_history_path.find('2018') + 4
        end_index = start_index + 5
        substring = train_user_history_path[start_index:end_index]
        print(substring)

        # 设置 CSV 文件路径
        csv_path = 'new_result//new_REMI-allfeature.csv'

        # 检查文件是否存在，如果存在则不写入表头
        try:
            with open(csv_path, 'r') as f:
                header_exists = True
        except FileNotFoundError:
            header_exists = False


        house_index,house_continuous_matrix,house_categorical_matrix1,house_categorical_matrix2,house_categorical_encodings,house_multi_matrix,num_house_categories_list2=read_house_host(house_path,house_id_column,'house',house_continuous_column,house_categorical_column1,house_categorical_column2)
        host_index,host_continuous_matrix,host_categorical_matrix1,host_categorical_matrix2,host_categorical_encodings,num_host_categories_list2=read_house_host(host_path,host_id_column,'host',host_continuous_column,host_categorical_column1,host_categorical_column2)

        #读取host图片特征
        host_pic_index, host_pic_continuous_matrix, host_pic_continuous_column = read_host_pic_features(host_pic_path)

        #读取正样本house和host动态特征
        comment_index,house_comment_matrix,host_comment_matrix,comment_house,comment_host=read_comment_features(comment_path,comment_id_column,house_comment_continuous_column,host_comment_continuous_column)


        #获取所有用户历史记录对应的house、host索引
        train_user_index, train_history_matrix_comment, train_history_matrix_host, train_history_matrix_host_pic, train_history_matrix_house, train_history_matrix_mask = user_history(train_user_history_path, comment_house, comment_host, comment_index, house_index, host_index, host_pic_index,max_history_len)
        val_user_index, val_history_matrix_comment, val_history_matrix_host, val_history_matrix_host_pic, val_history_matrix_house, val_history_matrix_mask = user_history(val_user_history_path, comment_house, comment_host, comment_index, house_index, host_index, host_pic_index,max_history_len)
        test_user_index, test_history_matrix_comment, test_history_matrix_host, test_history_matrix_host_pic, test_history_matrix_house, test_history_matrix_mask = user_history(test_user_history_path, comment_house, comment_host, comment_index, house_index, host_index, host_pic_index,max_history_len)


        house_scaler=creat_scaler(sclar_path,house_continuous_column)
        host_scaler=creat_scaler(sclar_path,host_continuous_column)
        house_comment_scaler=creat_scaler(sclar_path,house_comment_continuous_column)
        host_comment_scaler=creat_scaler(sclar_path,host_comment_continuous_column)
        host_pic_scaler=creat_scaler(sclar_path,host_pic_continuous_column)

        #连续特征标准化
        house_continuous_matrix = house_scaler.transform(house_continuous_matrix)
        host_continuous_matrix = host_scaler.transform(host_continuous_matrix)
        house_comment_matrix = house_comment_scaler.transform(house_comment_matrix)
        #     neg_house_comment_matrix = house_comment_scaler.transform(neg_house_comment_matrix)
        host_comment_matrix = host_comment_scaler.transform(host_comment_matrix)
        #     neg_host_comment_matrix = host_comment_scaler.transform(neg_host_comment_matrix)
        host_pic_continuous_matrix = host_pic_scaler.transform(host_pic_continuous_matrix)

        #特征矩阵转换成tensor
        house_continuous_tensor=torch.tensor(house_continuous_matrix).detach()
        house_categorical_tensor1=torch.tensor(house_categorical_matrix1).detach()
        house_categorical_tensor2=torch.tensor(house_categorical_matrix2).detach()
        house_multi_tensor=torch.tensor(house_multi_matrix).detach()
        house_comment_tensor=torch.tensor(house_comment_matrix).detach()
        #     neg_house_comment_tensor=torch.tensor(neg_house_comment_matrix).clone().detach()

        host_continuous_tensor=torch.tensor(host_continuous_matrix).detach()
        host_categorical_tensor1=torch.tensor(host_categorical_matrix1).detach()
        host_categorical_tensor2=torch.tensor(host_categorical_matrix2).detach()
        host_comment_tensor=torch.tensor(host_comment_matrix).detach()
        #     neg_host_comment_tensor=torch.tensor(neg_host_comment_matrix).clone().detach()
        host_pic_continuous_tensor=torch.tensor(host_pic_continuous_matrix).detach()

        #训练集输入特征编号、label.负样本动态特征（标准化后的tensor）
        train_pos_comment_id,train_house,train_host,train_host_pic,train_reviewer,train_label,train_house_comment_features,train_host_comment_features=get_train_input_index(train_neg_comment_path,comment_path,npratio,house_comment_scaler,host_comment_scaler,comment_index, house_index, host_index, host_pic_index,train_user_index,comment_house,comment_host,house_comment_tensor,host_comment_tensor)
        val_pos_comment_id,val_house,val_host,val_host_pic,val_reviewer,val_label,val_house_comment_features,val_host_comment_features=get_train_input_index(val_neg_comment_path,comment_path,npratio,house_comment_scaler,host_comment_scaler,comment_index, house_index, host_index, host_pic_index,val_user_index,comment_house,comment_host,house_comment_tensor,host_comment_tensor)
        test_pos_comment_id,test_house,test_host,test_host_pic,test_reviewer,test_label,test_house_comment_features,test_host_comment_features=get_train_input_index(test_neg_comment_path,comment_path,npratio,house_comment_scaler,host_comment_scaler,comment_index, house_index, host_index, host_pic_index,test_user_index,comment_house,comment_host,house_comment_tensor,host_comment_tensor)

        #获得训练集输入的house特征，shape(正样本数，6，特征维度)
        train_house_continuous_features,train_house_categorical_features1,train_house_categorical_feature2,train_house_multi_features=get_house_input(house_continuous_tensor,house_categorical_tensor1,house_categorical_tensor2,house_multi_tensor,train_house)
        #获得验证集输入的house特征，shape(正样本数，6，特征维度)
        val_house_continuous_features,val_house_categorical_features1,val_house_categorical_feature2,val_house_multi_features=get_house_input(house_continuous_tensor,house_categorical_tensor1,house_categorical_tensor2,house_multi_tensor,val_house)
        #获得测试集输入的house特征，shape(正样本数，6，特征维度)
        test_house_continuous_features,test_house_categorical_features1,test_house_categorical_feature2,test_house_multi_features=get_house_input(house_continuous_tensor,house_categorical_tensor1,house_categorical_tensor2,house_multi_tensor,test_house)


        #获得训练集输入的host特征
        train_host_continuous_features,train_host_categorical_features1,train_host_categorical_features2,train_host_pic_features=get_host_input(host_continuous_tensor,host_categorical_tensor1,host_categorical_tensor2,host_pic_continuous_tensor,train_host,train_host_pic)
        #获得验证集输入的host特征
        val_host_continuous_features,val_host_categorical_features1,val_host_categorical_features2,val_host_pic_features=get_host_input(host_continuous_tensor,host_categorical_tensor1,host_categorical_tensor2,host_pic_continuous_tensor,val_host,val_host_pic)
        #获得测试集输入的host特征
        test_host_continuous_features,test_host_categorical_features1,test_host_categorical_features2,test_host_pic_features=get_host_input(host_continuous_tensor,host_categorical_tensor1,host_categorical_tensor2,host_pic_continuous_tensor,test_host,test_host_pic)

        #user历史记录矩阵转换成tensor
        train_history_matrix_comment=torch.tensor(train_history_matrix_comment).detach()
        train_history_matrix_mask=torch.tensor(train_history_matrix_mask).detach()
        train_history_matrix_host=torch.tensor(train_history_matrix_host).detach()
        train_history_matrix_host_pic=torch.tensor(train_history_matrix_host_pic).detach()
        train_history_matrix_house=torch.tensor(train_history_matrix_house).detach()

        val_history_matrix_comment=torch.tensor(val_history_matrix_comment).detach()
        val_history_matrix_mask=torch.tensor(val_history_matrix_mask).detach()
        val_history_matrix_host=torch.tensor(val_history_matrix_host).detach()
        val_history_matrix_host_pic=torch.tensor(val_history_matrix_host_pic).detach()
        val_history_matrix_house=torch.tensor(val_history_matrix_house).detach()

        test_history_matrix_comment=torch.tensor(test_history_matrix_comment).detach()
        test_history_matrix_mask=torch.tensor(test_history_matrix_mask).detach()
        test_history_matrix_host=torch.tensor(test_history_matrix_host).detach()
        test_history_matrix_host_pic=torch.tensor(test_history_matrix_host_pic).detach()
        test_history_matrix_house=torch.tensor(test_history_matrix_house).detach()

        #得到训练集中user历史house、host对应编号，shape（正样本数，最大历史记录数）
        train_history_comment,train_history_host,train_history_host_pic,train_history_house,train_history_mask=get_user_input_index(train_history_matrix_comment,train_history_matrix_host,train_history_matrix_host_pic,train_history_matrix_house,train_history_matrix_mask,train_reviewer)
        #得到验证集中user历史house、host对应编号，shape（正样本数，最大历史记录数）
        val_history_comment,val_history_host,val_history_host_pic,val_history_house,val_history_mask=get_user_input_index(val_history_matrix_comment,val_history_matrix_host,val_history_matrix_host_pic,val_history_matrix_house,val_history_matrix_mask,val_reviewer)
        #得到测试集中user历史house、host对应编号，shape（正样本数，最大历史记录数）
        test_history_comment,test_history_host,test_history_host_pic,test_history_house,test_history_mask=get_user_input_index(test_history_matrix_comment,test_history_matrix_host,test_history_matrix_host_pic,test_history_matrix_house,test_history_matrix_mask,test_reviewer)

        #获得训练集输入user历史记录的house特征，shape(正样本数，最大历史记录数，特征维度)
        train_user_house_continuous_features,train_user_house_categorical_features1,train_user_house_categorical_feature2,train_user_house_multi_features=get_house_input(house_continuous_tensor,house_categorical_tensor1,house_categorical_tensor2,house_multi_tensor,train_history_house)
        #获得验证集输入user历史记录的house特征，shape(正样本数，最大历史记录数，特征维度)
        val_user_house_continuous_features,val_user_house_categorical_features1,val_user_house_categorical_feature2,val_user_house_multi_features=get_house_input(house_continuous_tensor,house_categorical_tensor1,house_categorical_tensor2,house_multi_tensor,val_history_house)
        #获得测试集输入user历史记录的house特征，shape(正样本数，最大历史记录数，特征维度)
        test_user_house_continuous_features,test_user_house_categorical_features1,test_user_house_categorical_feature2,test_user_house_multi_features=get_house_input(house_continuous_tensor,house_categorical_tensor1,house_categorical_tensor2,house_multi_tensor,test_history_house)

        #获得训练集输入user历史记录的host特征，shape(正样本数，最大历史记录数，特征维度)
        train_user_host_continuous_features,train_user_host_categorical_features1,train_user_host_categorical_features2,train_user_host_pic_features=get_host_input(host_continuous_tensor,host_categorical_tensor1,host_categorical_tensor2,host_pic_continuous_tensor,train_history_host,train_history_host_pic)
        #获得验证集输入user历史记录的host特征，shape(正样本数，最大历史记录数，特征维度)
        val_user_host_continuous_features,val_user_host_categorical_features1,val_user_host_categorical_features2,val_user_host_pic_features=get_host_input(host_continuous_tensor,host_categorical_tensor1,host_categorical_tensor2,host_pic_continuous_tensor,val_history_host,val_history_host_pic)
        #获得测试集输入user历史记录的host特征，shape(正样本数，最大历史记录数，特征维度)
        test_user_host_continuous_features,test_user_host_categorical_features1,test_user_host_categorical_features2,test_user_host_pic_features=get_host_input(host_continuous_tensor,host_categorical_tensor1,host_categorical_tensor2,host_pic_continuous_tensor,test_history_host,test_history_host_pic)

        #获得训练集输入user历史记录的comment特征，shape(正样本数，最大历史记录数，特征维度)
        train_user_house_comment_features,train_user_host_comment_features=get_history_comment_input(train_history_comment,house_comment_tensor,host_comment_tensor)
        val_user_house_comment_features,val_user_host_comment_features=get_history_comment_input(val_history_comment,house_comment_tensor,host_comment_tensor)
        test_user_house_comment_features,test_user_host_comment_features=get_history_comment_input(test_history_comment,house_comment_tensor,host_comment_tensor)    


        num_house_categorical2_features=house_categorical_matrix2.shape[1]
        num_host_categorical2_features=host_categorical_matrix2.shape[1]
        num_house_categorical1_features=house_categorical_matrix1.shape[1]
        num_host_categorical1_features=host_categorical_matrix1.shape[1]
        num_house_multi_features=house_multi_matrix.shape[1]
        num_house_continuous_features=house_continuous_matrix.shape[1]
        num_host_continuous_features=host_continuous_matrix.shape[1]   
        num_house_comment_features=house_comment_matrix.shape[1]
        num_host_comment_features=host_comment_matrix.shape[1]
        num_host_pic_continuous_features=host_pic_continuous_matrix.shape[1]


        train_modals= f'model//new_REMI-allfeature+{substring}+{n}.pth'
        #训练集
        train_dataset = TensorDataset(train_house_continuous_features,train_house_categorical_features1,train_house_categorical_feature2,train_house_multi_features,train_house_comment_features,
            train_host_continuous_features,train_host_categorical_features1,train_host_categorical_features2,train_host_comment_features,train_host_pic_features,
            train_user_house_continuous_features,train_user_house_categorical_features1,train_user_house_categorical_feature2,train_user_house_multi_features,train_user_house_comment_features,
            train_user_host_continuous_features,train_user_host_categorical_features1,train_user_host_categorical_features2,train_user_host_comment_features,train_user_host_pic_features,
            train_label,train_history_mask)
        val_dataset = TensorDataset(val_house_continuous_features, val_house_categorical_features1, val_house_categorical_feature2, val_house_multi_features, val_house_comment_features,
            val_host_continuous_features, val_host_categorical_features1, val_host_categorical_features2, val_host_comment_features, val_host_pic_features,
            val_user_house_continuous_features, val_user_house_categorical_features1, val_user_house_categorical_feature2, val_user_house_multi_features, val_user_house_comment_features,
            val_user_host_continuous_features, val_user_host_categorical_features1, val_user_host_categorical_features2, val_user_host_comment_features, val_user_host_pic_features,
            val_label, val_history_mask)


        # 创建数据加载器

        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


        # 确保您的计算机上有CUDA支持的GPU
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        # 创建大模型的实例
        model = MatchingModel(num_house_categorical1_features,num_host_categorical1_features,num_house_categorical2_features,num_host_categorical2_features,num_house_multi_features,multi_embedding_dim,
                     num_house_categories_list2,num_host_categories_list2,num_house_continuous_features,num_host_continuous_features,
                     num_host_pic_continuous_features,num_house_comment_features,feature_dim,num_heads,max_history_len,
                     num_experts, num_tasks, expert_hidden_units, gate_hidden_units,user_dnn_hidden_units,house_dnn_hidden_units,attetion_dnn_hidden_units,device)


        model.to(device)

        #进一步处理 列表转移到GPU
        for i in range(len(model.house_embedding.house_categorical1_embeddings)):
            model.house_embedding.house_categorical1_embeddings[i] = model.house_embedding.house_categorical1_embeddings[i].to(device)
        for i in range(len(model.house_embedding.house_categorical2_embeddings)):
            model.house_embedding.house_categorical2_embeddings[i] = model.house_embedding.house_categorical2_embeddings[i].to(device)
        for i in range(len(model.house_embedding.host_categorical1_embeddings)):
            model.house_embedding.host_categorical1_embeddings[i] = model.house_embedding.host_categorical1_embeddings[i].to(device)
        for i in range(len(model.house_embedding.host_categorical2_embeddings)):
            model.house_embedding.host_categorical2_embeddings[i] = model.house_embedding.host_categorical2_embeddings[i].to(device)
        for i in range(len(model.house_embedding.multi_embedding_layer)):
            model.house_embedding.multi_embedding_layer[i] = model.house_embedding.multi_embedding_layer[i].to(device)

        for i in range(len(model.house_embedding.house_continuous_embedding_layer)):
            model.house_embedding.house_continuous_embedding_layer[i] = model.house_embedding.house_continuous_embedding_layer[i].to(device)
        for i in range(len(model.house_embedding.house_comment_embedding_layer)):
            model.house_embedding.house_comment_embedding_layer[i] = model.house_embedding.house_comment_embedding_layer[i].to(device)
        for i in range(len(model.house_embedding.host_continuous_embedding_layer)):
            model.house_embedding.host_continuous_embedding_layer[i] = model.house_embedding.host_continuous_embedding_layer[i].to(device)
        for i in range(len(model.house_embedding.host_pic_embedding_layer)):
            model.house_embedding.host_pic_embedding_layer[i] = model.house_embedding.host_pic_embedding_layer[i].to(device)


        rec_criterion = nn.BCELoss(size_average=True, reduce=True)
        optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)

        #训练
        model_training(model,train_loader,val_loader,rec_criterion,optimizer,500,device)
        with open(train_modals, 'wb') as f:
            cloudpickle.dump(model, f)     
        #测试
        test_reviewer=torch.tensor(test_reviewer).detach()
        test_host=torch.tensor(test_host).detach()
        test_house=torch.tensor(test_house).detach()
        test_pos_comment_id=torch.tensor(test_pos_comment_id, dtype=torch.int64).detach()
        test_dataset = TensorDataset(test_pos_comment_id,test_house,test_host,test_reviewer,test_house_continuous_features, test_house_categorical_features1, test_house_categorical_feature2, test_house_multi_features, test_house_comment_features,
            test_host_continuous_features, test_host_categorical_features1, test_host_categorical_features2, test_host_comment_features, test_host_pic_features,
            test_user_house_continuous_features, test_user_house_categorical_features1, test_user_house_categorical_feature2, test_user_house_multi_features, test_user_house_comment_features,
            test_user_host_continuous_features, test_user_host_categorical_features1, test_user_host_categorical_features2, test_user_host_comment_features, test_user_host_pic_features,
            test_label, test_history_mask)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
        df_results,average_test_loss,average_auc_test=test_model(model, test_loader)
        #测试的每个样本结果保存到csv
    #     file_name = f"result/REMI{n}.csv"
    #     df_results.to_csv(file_name, index=False)

        top_5_ndcg,top_5_hr=top_evaluation(df_results,5)
        top_10_ndcg,top_10_hr=top_evaluation(df_results,10)   

        new_row = {
            '实验数': n, 
            '测试集总损失': average_test_loss, 
            'AUC': average_auc_test, 
            'NDCG@5': top_5_ndcg, 
            'HR@5': top_5_hr, 
            'NDCG@10': top_10_ndcg, 
            'HR@10': top_10_hr
            }



        # 将新的行追加到 CSV 文件中
        pd.DataFrame([new_row]).to_csv(csv_path, mode='a', index=False, header=not header_exists)
        header_exists = True  # 确保后续追加时不再写入表头

i=+1
01.cs


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

Epoch 5,loss:2.6763582814414546
Validation Loss: 3.082756280899048,AUC: 0.8472622282608695
Epoch 10,loss:2.6151915361296454
Validation Loss: 3.0723340096681016,AUC: 0.8521923524844719
Epoch 15,loss:2.5681390132544175
Validation Loss: 2.9935224367224653,AUC: 0.8556579968944098
Epoch 20,loss:2.5501134035722264
Validation Loss: 3.113721391429072,AUC: 0.8547457298136645
Epoch 25,loss:2.523020820797614
Validation Loss: 2.9795759553494663,AUC: 0.8545108695652176
Epoch 30,loss:2.5020029679784237
Validation Loss: 3.01599439330723,AUC: 0.8546399456521739
Epoch 35,loss:2.49295888756806
Validation Loss: 3.047296917956808,AUC: 0.8557822204968943
Epoch 40,loss:2.4774825617952168
Validation Loss: 3.021443854207578,AUC: 0.8579677795031059
早停策略触发，停止训练在第 39 个epoch.
Test Loss: 3.173420991897583,AUC: 0.8405363636363634,rec_loss: 0.0
i=+2
01.cs


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

Epoch 5,loss:2.643219956811869
Validation Loss: 3.0498955042465874,AUC: 0.847710597826087
Epoch 10,loss:2.582390110447722
Validation Loss: 3.0092870152514912,AUC: 0.853131793478261
Epoch 15,loss:2.526746614924017
Validation Loss: 3.0311536374299424,AUC: 0.8540130046583849
Epoch 20,loss:2.472195377889669
Validation Loss: 3.0630590086397915,AUC: 0.8566013198757766
Epoch 25,loss:2.477025279458964
Validation Loss: 2.99752604443094,AUC: 0.8561607142857143
Epoch 30,loss:2.4500949337797344
Validation Loss: 2.998241072115691,AUC: 0.8545273680124225
Epoch 35,loss:2.458918845878457
Validation Loss: 3.007676311161207,AUC: 0.8605939440993788
Epoch 40,loss:2.448074849146717
Validation Loss: 2.9393778987552808,AUC: 0.8615663819875776
Epoch 45,loss:2.4378966565402047
Validation Loss: 2.9235320713209068,AUC: 0.8669934006211181
Epoch 50,loss:2.425081896332075
Validation Loss: 2.943773435509723,AUC: 0.8599369177018632
Epoch 55,loss:2.44379689558497
Validation Loss: 2.965805706770524,AUC: 0.8551339285714

...

Epoch 5,loss:3.1338259214308204
Validation Loss: 3.2275994746915755,AUC: 0.8190950776947287
Epoch 10,loss:3.0667185405405557
Validation Loss: 3.2201342351974978,AUC: 0.8276950973642802
Epoch 15,loss:3.0190217611266346
Validation Loss: 3.150061345869495,AUC: 0.8326851396538159
Epoch 20,loss:2.99222022149621
Validation Loss: 3.12899887177252,AUC: 0.8383407503933911
Epoch 25,loss:2.9759772492618097
Validation Loss: 3.100742224724062,AUC: 0.8422383949645948
Epoch 30,loss:2.9529674460248247
Validation Loss: 3.1155083487110753,AUC: 0.8387653668371361
Epoch 35,loss:2.9215504803308625
Validation Loss: 3.1040511208195842,AUC: 0.844398480527144
Epoch 40,loss:2.9140134381084906
Validation Loss: 3.0556158404196463,AUC: 0.8460659667584581
Epoch 45,loss:2.898041597226771
Validation Loss: 3.04823717763347,AUC: 0.8492660798583792
Epoch 50,loss:2.905477805835445
Validation Loss: 3.100134426547635,AUC: 0.8435365853658537
Epoch 55,loss:2.8939465197121224
Validation Loss: 3.0538921202382734,AUC: 0.8504195

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

Epoch 5,loss:3.2022629481990164
Validation Loss: 3.309274834971274,AUC: 0.8032143735247838
Epoch 10,loss:3.1022058899809672
Validation Loss: 3.351784844552317,AUC: 0.8108955792682926
Epoch 15,loss:3.089089350002568
Validation Loss: 3.213211928644488,AUC: 0.823159421715185
Epoch 20,loss:3.0726603531255954
Validation Loss: 3.1529581546783447,AUC: 0.831661646833202
Epoch 25,loss:3.0546892968619743
Validation Loss: 3.1367325090592906,AUC: 0.8353607518686074
Epoch 30,loss:3.0320656532194556
Validation Loss: 3.134968465374362,AUC: 0.8372037273800157
Epoch 35,loss:3.04291175632942
Validation Loss: 3.137362572454637,AUC: 0.8345050649095201
Epoch 40,loss:3.0155556201934814
Validation Loss: 3.150039365214686,AUC: 0.8330773013375297
Epoch 45,loss:3.0139776875333086
Validation Loss: 3.1214681517693306,AUC: 0.8382083497246264
Epoch 50,loss:3.003217999528094
Validation Loss: 3.1871772735349593,AUC: 0.8368064024390244
Epoch 55,loss:2.994724131212002
Validation Loss: 3.09471111143789,AUC: 0.8399438188

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

Epoch 5,loss:3.1130118457282463
Validation Loss: 3.1960640645796254,AUC: 0.8249055861526357
Epoch 10,loss:3.041116295791254
Validation Loss: 3.12878502568891,AUC: 0.8363181058221874
Epoch 15,loss:3.0001608162391475
Validation Loss: 3.1472859459538616,AUC: 0.8336591512588511
Epoch 20,loss:2.981671696755944
Validation Loss: 3.1225612009725263,AUC: 0.8348491591266719
Epoch 25,loss:2.96081656944461
Validation Loss: 3.0842100420305805,AUC: 0.842974036191975
